# Callbacks

Quoting from the Gurobi [documentation](https://www.gurobi.com/documentation/8.1/refman/py_callbacks.html)

> A callback is a **user function** that is **called periodically** by the Gurobi optimizer in order to allow the user to **query or modify** the state of the optimization.

That is, we create our own function which we will call *the callback* and we pass it to the ``optimie`` method. The callback will then be called at specific points while the problem is being solved. We decide at which points the callback should be called, and within the callback we can use specific methods to query details on the state of the optimization procedure, and we can also modify what the optimization algorithm is doing. ¨

Let us build and example MIP to which we will attach a callback.

In [27]:
# We import the necessary stuff and build a toy MIP
from gurobipy.gurobipy import Model, GRB, Column
import random as r

def build_model():
    r.seed(1)
    
    m = Model()

    # Creates 3 integer variables. 
    x = m.addVars(300, vtype=GRB.INTEGER, name="x")
    # The variables should be appended as attributes 
    # with a leading _ to the model object in order 
    # to be visible in the callback.
    m._x = x

    # Creates the objective
    expr = x.prod({i:(10 + r.random()*20) for i in range(300)})
    m.setObjective(expr, GRB.MINIMIZE)

    # Creates the constraints 
    lhs1 = x.prod({i:(-20 + r.random()*40) for i in range(300)})
    c1 = m.addConstr(lhs1, GRB.GREATER_EQUAL, 120, "c1")

    # Creates the constraints 
    lhs2 = x.prod({i:(-5 + r.random()*10) for i in range(300)})
    c2 = m.addConstr(lhs2, GRB.GREATER_EQUAL, 70, "c2")
    
    # Finally it retuns the model
    return m

A callback is created by defining a function. The function takes as argument the model where we want to use the callback and a ``where`` variable. The where variable is used to check whether the optimizer reached the intended point during the execution. The basic structure of a callback is the following

In [ ]:
# Basic structure of a callback
def my_callback(model, where):
    
    # 1. Check if we are at the intended where
    
    # 2. Execute the code of the callback 
    # (e.g., query details or modify execution)

To tell Gurobi that we want to use the callback then we simply pass the callback to the ``optimize`` method as follows

In [ ]:
m.optimize(my_callback)

Now, during the course of the optimization, Gurobi will periodically call our function ``my_callback`` passing as arguments the focal ``model`` (``m`` in our case) and a ``where``. The ``where`` takes a value which corresponds to a different stage in the course of the algorithm used to solve the problem. The possible values are listed at the [Callback Coded page](https://www.gurobi.com/documentation/8.1/refman/callback_codes.html#sec:CallbackCodes). As an example, if Gurobi is currently executing a presolve it will pass a value of ``1`` (corresponding to ``GRB.Callback.PRESOLVE`` in Python) for the ``where``, and if it has just found an integer solution during the course of the Branch & Bound method it will pass a value of ``4`` (corresponding to ``GRB.Callback.MIPSOL`` in Python). 
In this way the callback is able to understand at what stage of the algorithm the solver is, and execute accordingly.

As an example, let us pass a very simple callback to our solver which simply gives us basic information.

In [28]:
# We build a model using the function we created above
m = build_model()

# We create
def my_callback(model,where):
    if where == GRB.Callback.MIPNODE:
        print(">>> We are at a B&C node")
    if where == GRB.Callback.MIPSOL:
        print("We found an integer solution!")


m.setParam(GRB.Param.OutputFlag,0) # We tell Gurobi not to print the default output
m.optimize(my_callback)

We found an integer solution!
We found an integer solution!
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a B&C node
>>> We are at a

Let us now try to get some more useful information about what is happening during the solution of our problem.
In order to retrieve information during the execution of an algorithm (e.g., Branch & Bound), a Model object
has a few methods: 
+ [``cbGet``](https://www.gurobi.com/documentation/8.1/refman/py_model_cbget.html), which can be used to query one of the possible ``what`` information listed [here](https://www.gurobi.com/documentation/8.1/refman/callback_codes.html#sec:CallbackCodes).
+ [``cbGetSolution``](https://www.gurobi.com/documentation/8.1/refman/py_model_cbgetsolution.html), which can be used only when ``where==GRB.Callback.MIPSOL`` to retrieve the newly found integer solution
+ [``cbGetNodeRel``](https://www.gurobi.com/documentation/8.1/refman/py_model_cbgetnoderel.html), which can be used only when ``where==GRB.Callback.MIPNODE`` and ``GRB.Callback.MIPNODE_STATUS==GRB.OPTIMAL`` to retrieve the value of the node relaxation solution at the current node.

Let us see some examples.

In [30]:
# We re-build a model using the function we created above
m = build_model()

# We create
def my_callback(model,where):
    if where == GRB.Callback.MIPNODE:
        print(">>> We are at a B&C node ",model.cbGet(GRB.Callback.MIPNODE_NODCNT))
        print("Status ",model.cbGet(GRB.Callback.MIPNODE_STATUS))
        # Status codes are reported here https://www.gurobi.com/documentation/8.1/refman/optimization_status_codes.html#sec:StatusCodes
        print("Best objective ", model.cbGet(GRB.Callback.MIPNODE_OBJBST))
        print("Best bound ", model.cbGet(GRB.Callback.MIPNODE_OBJBND))

m.setParam(GRB.Param.OutputFlag,0) # We tell Gurobi not to print the default output
m.optimize(my_callback)

>>> We are at a B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  155.85456572544138
>>> We are at a B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  156.1850685876831
>>> We are at a B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  158.52942228414668
>>> We are at a B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  159.19229747204184
>>> We are at a B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  159.68656909581023
>>> We are at a B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  159.74240365603987
>>> We are at a B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  160.0967536635286
>>> We are at a B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  160.1265439637806
>>> We are at a B&C node  0.0
Status  2
Best objective  163.26602804460956
Best bound  160.15457676303515
>>> We are at a B&C node  0.0
Status  2
Best obje

In [ ]:
If both x1 and x3 are positive we force one of them to be 0